<a href="https://colab.research.google.com/github/livjab/DS3-Kaggle-Comptetion/blob/master/LJ_DS3_Kaggle_Competition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np

In [2]:
test = pd.read_csv("https://raw.githubusercontent.com/livjab/DS3-Kaggle-Comptetion/master/test_features.csv")
train = pd.read_csv("https://raw.githubusercontent.com/livjab/DS3-Kaggle-Comptetion/master/train_features.csv")
train_y = pd.read_csv("https://raw.githubusercontent.com/livjab/DS3-Kaggle-Comptetion/master/train_labels.csv")

train.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [3]:
train.shape, train_y.shape, test.shape

((59400, 40), (59400, 2), (14358, 40))

In [4]:
train.dtypes

id                         int64
amount_tsh               float64
date_recorded             object
funder                    object
gps_height                 int64
installer                 object
longitude                float64
latitude                 float64
wpt_name                  object
num_private                int64
basin                     object
subvillage                object
region                    object
region_code                int64
district_code              int64
lga                       object
ward                      object
population                 int64
public_meeting            object
recorded_by               object
scheme_management         object
scheme_name               object
permit                    object
construction_year          int64
extraction_type           object
extraction_type_group     object
extraction_type_class     object
management                object
management_group          object
payment                   object
payment_ty

In [5]:
# fast first baseline

train_y["status_group"].value_counts(normalize=True)

functional                 0.543081
non functional             0.384242
functional needs repair    0.072677
Name: status_group, dtype: float64

In [6]:
# going to combine train_y with train for ease

train = pd.merge(train, train_y)
train.shape

(59400, 41)

In [7]:
# starting with a simple logistic regression model using only numeric columns
# and simple inputer to handle nulls

numeric_columns = train.select_dtypes(include="number").columns
numeric_columns.tolist()

['id',
 'amount_tsh',
 'gps_height',
 'longitude',
 'latitude',
 'num_private',
 'region_code',
 'district_code',
 'population',
 'construction_year']

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer

features = numeric_columns
target = "status_group"

X_train = train[features]
y_train = train[target]
X_test = test[features]

imputer = SimpleImputer()
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

log_reg = LogisticRegression(solver="lbfgs")
log_reg.fit(X_train_imputed, y_train)

log_reg.score(X_train_imputed, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.550993265993266

In [0]:
# going to submit baseline

y_pred = log_reg.predict(X_test_imputed)

sample_submission = pd.read_csv('https://raw.githubusercontent.com/livjab/DS3-Kaggle-Comptetion/master/sample_submission.csv')
submission = sample_submission.copy()
submission['status_group'] = y_pred

from google.colab import files
submission.to_csv('LJ-first-submission.csv', index=False)
files.download('LJ-first-submission.csv')

In [10]:
# let's up the ante with xgboost classifier

from xgboost import XGBClassifier
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score

scores = cross_validate(XGBClassifier(max_depth=8, n_estimators=100),
                       X_train_imputed,
                       y_train,
                       scoring="accuracy",
                       cv=3,
                       return_train_score=True,
                       return_estimator=True)

pd.DataFrame(scores)


,fit_time,score_time,estimator,test_score,train_score
0,19.033911,0.588790,"XGBClassifier(base_score=0.5, booster='gbtree'...",0.690859,0.758889
1,18.978579,0.647721,"XGBClassifier(base_score=0.5, booster='gbtree'...",0.692121,0.760101
2,18.979190,0.597876,"XGBClassifier(base_score=0.5, booster='gbtree'...",0.687626,0.759419


In [0]:
boost = XGBClassifier(max_depth=8, n_estimators=100)
boost.fit(X_train_imputed, y_train)

y_pred = boost.predict(X_test_imputed)

In [0]:
sample_submission = pd.read_csv('https://raw.githubusercontent.com/livjab/DS3-Kaggle-Comptetion/master/sample_submission.csv')
submission = sample_submission.copy()
submission['status_group'] = y_pred

from google.colab import files
submission.to_csv('LJ-second-submission.csv', index=False)
files.download('LJ-second-submission.csv')

In [13]:
!pip install category_encoders

     |████████████████████████████████| 92kB 5.9MB/s 


In [0]:
# Would like to create a model that includes all features
# Will need to do some serious cleaning/encoding of categorical features

In [0]:
# drop "id" because it is random

X_train = train.drop(columns="id")
X_test = test.drop(columns="id")

In [16]:
train.amount_tsh.isnull().sum()

0

In [34]:
train.date_recorded.dtypes

dtype('O')

In [18]:
train.date_recorded.describe()

count          59400
unique           356
top       2011-03-15
freq             572
Name: date_recorded, dtype: object

In [19]:
train.date_recorded.isnull().sum()

0

In [40]:
# Want to split the date recorded into year, month, and day categories

train["date_recorded_sep"] = train["date_recorded"].str.split("-")
train["date_recorded_sep"].head()

0    [2011, 03, 14]
1    [2013, 03, 06]
2    [2013, 02, 25]
3    [2013, 01, 28]
4    [2011, 07, 13]
Name: date_recorded_sep, dtype: object

In [0]:
train["year_recorded"] = train["date_recorded_sep"].str[0]
train["month_recorded"] = train["date_recorded_sep"].str[1]
train["day_of_month_recorded"] = train["date_recorded_sep"].str[2]


In [0]:
# Let's turn these into numeric columns

train["year_recorded"] = pd.to_numeric(train["year_recorded"])
train["month_recorded"] = pd.to_numeric(train["month_recorded"])
train["day_of_month_recorded"] = pd.to_numeric(train["day_of_month_recorded"])

In [0]:
# same for test set

test["date_recorded_sep"] = test["date_recorded"].str.split("-")

test["year_recorded"] = test["date_recorded_sep"].str[0]
test["month_recorded"] = test["date_recorded_sep"].str[1]
test["day_of_month_recorded"] = test["date_recorded_sep"].str[2]

test["year_recorded"] = pd.to_numeric(test["year_recorded"])
test["month_recorded"] = pd.to_numeric(test["month_recorded"])
test["day_of_month_recorded"] = pd.to_numeric(test["day_of_month_recorded"])


In [0]:
train = train.drop(columns="date_recorded_sep")
test = test.drop(columns="date_recorded_sep")

In [60]:
train.funder.value_counts().head(10)

Government Of Tanzania    9084
Danida                    3114
Hesawa                    2202
Rwssp                     1374
World Bank                1349
Kkkt                      1287
World Vision              1246
Unicef                    1057
Tasaf                      877
District Council           843
Name: funder, dtype: int64

In [59]:
train.funder.isnull().sum()

3635

In [0]:
train["funder"] = train.funder.fillna(value="Government Of Tanzania")
test["funder"] = test.funder.fillna(value="Government of Tanzania")

In [62]:
# high cardinality so trying binary encoding

#encoder = ce.BinaryEncoder()
#encoded = encoder.fit_transform(train[["funder"]])
#print(f'{len(encoded.columns)} columns')
#encoded.head()

12 columns


,funder_0,funder_1,funder_2,funder_3,funder_4,funder_5,funder_6,funder_7,funder_8,funder_9,funder_10,funder_11
0,0,0,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0,1,1
3,0,0,0,0,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,0,0,0,1,0,1


In [0]:
import category_encoders as ce

X_train = train.drop(columns=["status_group", "date_recorded"])
y_train = train["status_group"]
X_test = test.drop(columns="date_recorded")

imputer = SimpleImputer(strategy="most_frequent")
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

encoder = ce.BinaryEncoder()
X_train = encoder.fit_transform(X_train)
X_test = encoder.transform(X_test)


In [85]:
scores = cross_validate(XGBClassifier(max_depth=5, n_estimators=100),
                       X_train,
                       y_train,
                       scoring="accuracy",
                       cv=3,
                       return_train_score=True,
                       return_estimator=True)

pd.DataFrame(scores)

,fit_time,score_time,estimator,test_score,train_score
0,104.065572,0.700282,"XGBClassifier(base_score=0.5, booster='gbtree'...",0.772273,0.790404
1,103.639979,0.708248,"XGBClassifier(base_score=0.5, booster='gbtree'...",0.775808,0.788763
2,103.109728,0.718316,"XGBClassifier(base_score=0.5, booster='gbtree'...",0.773636,0.792525


In [0]:
boost = XGBClassifier(max_depth=5, n_estimators=100)
boost.fit(X_train, y_train)

y_pred = boost.predict(X_test)

In [0]:
# third submission

sample_submission = pd.read_csv('https://raw.githubusercontent.com/livjab/DS3-Kaggle-Comptetion/master/sample_submission.csv')
submission = sample_submission.copy()
submission['status_group'] = y_pred

from google.colab import files
submission.to_csv('LJ-third-submission.csv', index=False)
files.download('LJ-third-submission.csv')